##**Connect to T4 GPU Runtime.

##1.Mount your google drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##2.Navigate to where you have saved the repository on your drive

In [ ]:
# Navigate to the project directory
%cd /content/drive/My Drive/Wave-U-Net-Pytorch

##3.Install the dependencies

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install gradio pydub

##4.Run this cell to execute the applicaton.

In [14]:
import gradio as gr
import soundfile as sf
import os
import numpy as np
import subprocess
import torch

def process_audio(audio):
    if audio is None:
        return None, None, None, None

    audio_path = audio  # Unpack the audio tuple from Gradio
    audio_data, sample_rate = sf.read(audio_path)

    if len(audio_data.shape) > 1:
        audio_data = np.mean(audio_data, axis=1)

    # Define the directory to save the uploaded audio
    save_dir = "uploads"
    os.makedirs(save_dir, exist_ok=True)

    # Define the path for the saved file
    file_path = os.path.join(save_dir, "uploaded_audio.wav")

    sf.write(file_path, audio_data, sample_rate)

    # Construct the command to execute
    command = [
        "python", "predict.py",
        "--load_model", "checkpoints/waveunet/model",
        "--input", "uploads/uploaded_audio.wav"
    ]

    # Check if CUDA is available
    if torch.cuda.is_available():
        command.append("--cuda")

    # Run the command and wait for completion
    try:
        subprocess.run(command, check=True)  # Wait until completion
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")
        return None, None, None, None

    # After subprocess completes, process the output files
    result_directory = "uploads"

    # Get all .wav files in the result directory
    wav_files = [file for file in os.listdir(result_directory) if file.endswith(".wav")]

    # Exclude the file named "uploaded_audio.wav"
    filtered_files = [os.path.join(result_directory, file) for file in wav_files if file != "uploaded_audio.wav"]

    # Ensure we have at least four output files (Bass, Drums, Others, Vocals)
    while len(filtered_files) < 4:
        filtered_files.append(None)

    # Return the audio paths for the outputs
    return filtered_files[0], filtered_files[1], filtered_files[2], filtered_files[3]

# Define the Gradio interface with a title
demo = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath", label='Input Audio'),
    outputs=[
        gr.Audio(type="filepath", label='Bass'),
        gr.Audio(type="filepath", label='Drums'),
        gr.Audio(type="filepath", label='Others'),
        gr.Audio(type="filepath", label='Vocals')
    ],
    title="Vocal Unravel: Audio Source Separation"
)

# Launch the Gradio interface
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://73f15cf38c666214bd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
